In [ ]:
pip install azure-ai-ml

In [ ]:
pip show azure-ai-ml

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
%%writefile src/diabetes-training.py
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

# 1.Pandas (pd) y Numpy (np): Para manejo y procesamiento de datos.

# 2.train_test_split: Divide los datos en conjuntos de entrenamiento y prueba.

# 3.LogisticRegression: Modelo para clasificación binaria.

# 4.roc_auc_score y roc_curve: Métricas para evaluar el modelo.

In [ ]:
# Cargar el dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# Carga un archivo CSV llamado diabetes.csv que contiene datos para predecir si una persona tiene diabetes (variable objetivo: Diabetic).

In [ ]:
# Separar características (X) y etiquetas (y)
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values
# Selecciona las columnas relevantes para las características (X) y la variable objetivo (y).


In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Divide el conjunto de datos en: 70% entrenamiento (X_train, y_train) 30% prueba (X_test, y_test).


In [ ]:
# Entrenar el modelo de regresión logística
reg = 0.01
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# C=1/reg: Parámetro inverso de regularización (penalización por sobreajuste). Un valor más alto de reg implica más regularización.
# solver="liblinear": Método usado para optimizar el modelo.

In [ ]:
# Calcular precisión (accuracy)
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# Predice etiquetas para los datos de prueba y calcula la proporción de predicciones correctas.

In [ ]:
# Calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print('AUC:', auc)

# Obtiene las probabilidades de cada clase.
# Calcula el área bajo la curva ROC (AUC), una métrica que mide la capacidad del modelo para clasificar correctamente.

2. Configurar y ejecutar un trabajo en Azure ML

In [ ]:
from azure.ai.ml import command

# Configurar el trabajo
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-curated-env",
    experiment_name="diabetes-training"
)

# command: Define un trabajo que ejecutará diabetes-training.py en un entorno de Azure ML.
# Parámetros clave:
# code="./src": Directorio que contiene el script.
# command="python diabetes-training.py": Comando que ejecutará el script.
# environment: Entorno preconfigurado de Azure ML con scikit-learn.
# compute="aml-cluster": Cluster de computación en Azure ML.

In [ ]:
# Enviar el trabajo
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

# Envia el trabajo al cluster y proporciona un enlace para monitorear el progreso en Azure Machine Learning Studio.

3. Trabajar con entornos personalizados

Listar entornos existentes

In [ ]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name)

# Lista todos los entornos disponibles en tu cuenta de Azure ML.

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env.description, env.tags)

# Recupera información sobre un entorno específico basado en su nombre y versión.

Crear un entorno desde una imagen Docker

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

# Define un nuevo entorno personalizado basado en una imagen Docker.


Usar el entorno personalizado

In [ ]:
from azure.ai.ml import command

In [ ]:
# configurar trabajo

job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# Enviar trabajo:

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

# Usa el entorno docker-image-example:1 para ejecutar el trabajo en Azure ML.